One of the best things about Excel is its ability for end users to engage and interact with the data. However we as dsasdfasd may not want to do it.

Let's build a little model. 

- Our raw data will go in a protected worksheet called `data`. The only thing end-users can do with this worksheet is use the data filters. 
- We will add a `summary` worksheet that end-users can use to find the total sales for a region. We will protect all of this worksheet except for the cells they are meant to interact with.
- We will use data validation to guide the values that our end users can input. 
- We also would like the `summary` worksheet to be what the users see when they open the workbook.

Let's get started.

In [47]:
# Add in our data
import xlsxwriter
regions = ['C','B','B','B','C','A','C','B','B','C']
sales = [201,293,431,111,241,321,311,187,400,354]
labels = ['Region','Sales']

workbook = xlsxwriter.Workbook('sales-model.xlsx')
worksheet = workbook.add_worksheet('data')

worksheet.write_column('A2',regions)
worksheet.write_column('B2',sales)
worksheet.write_row('A1',labels)



0

## Adding an AutoFilter

We can apply an AutoFilter to our data sheet with the `autofilter()` method:

In [48]:
# Add an autofilter to the data
worksheet.autofilter('A1:B11')

## Worksheet protection

Let's protect this `data` worksheet so that all users can do with it is manipulate the AutoFilter. 

We can do so with the `protect()` method. There are several optional setting to choose from in this method: we will set `autofilter` to `True` by passing it into a dictionary. 

To see the all the options for worksheet protection, check out Chapter 7 of the [`xlsxwriter` documentation](https://github.com/jmcnamara/XlsxWriter/blob/master/docs/XlsxWriter.pdf).

In [49]:
# Protect the worksheet but allow autofiltering
worksheet.protect('data', {'autofilter': True})

### Protecting cells and ranges

Now let's move to our `summary` worksheet. Here, we will allow our users to make some changes to the worksheet, via data validation.

In [50]:
# Add the summary worksheet
worksheet2 = workbook.add_worksheet('summary')

To protect only *some* cells in a worksheet, we can take the following steps:

1. Protect the entire worksheet with the `protect()` method
2. Create a cell format with *unlocked* cells by setting `locked` to `False`
3. Pass that format into any cells which we *do* want users to interact with

Let's try it out:

In [51]:
# Protect the whole worksheet
worksheet2.protect()

# Add an unlocked format
unlocked = workbook.add_format({'locked':False})

# This cell remains locked...
worksheet2.write('A1','Select region: ')

# This cell will get unlocked so that users can 
# interact with it
worksheet2.write('B1','A',unlocked)

# This will conditionally sum off the value of B1
worksheet2.write_formula('C1','=SUMIF(data!A:A,B1,data!B:B)')

# Widen the columns for formatting
worksheet2.set_column('A:C',12)

0

## Data validation

You may have noticed that we passed the value of `A` into cell `B1`. Then cell `C1` conditionally sums that value.

What if our end-user wanted to know the sum of regions `B` or `C`? Let's make that easier for them by providing a drop-down list. This is a form of *data validation*.

Like most things in `xlsxwriter`, we will just scratch the surface of what's possible. For more on data validation, check out Chapter 21 of the documentation. 

Let's use the `data_validation()` method to add a drop-down to cell `B1`.

In [52]:
# Add data validation to B1
# We are validating from a list
worksheet.data_validation('A1', {'validate':'list',
# Here are the values we would allow in the list
                                'source':['A','B','C'],
# We would like to add a dropdown to this validation
                                'dropdown':True})

Let's do one thing to make this validation a little more dynamic. Rather than "hard-coding" our possible value of `A`,`B` and `C` into the list, we can find the sorted, unique values of the `regions` list.

In [53]:
# This code will give us all the unique values of our list
region_unique = list(set(regions))

# Now we will sort the result
region_unique.sort()

print(region_unique)

['A', 'B', 'C']


In [54]:
# Now a more dynamic data drop-down
worksheet.data_validation('A1', {'validate':'list',
                                'source':region_unique,
                                'dropdown':True})

As mentioned before, there are a *lot* of ways to do data validation. Let's try one more in cell `A2`:

In [55]:
# Now a more dynamic data drop-down
worksheet2.data_validation('A2', {'validate':'list',
                                'source':region_unique,
                                'input_title':'Enter a region',
                                'input_message': 'Value should be alphabetical'})

# Make sure this is unlocked for our users!
worksheet2.write('A2','B',unlocked)

0

## Setting the active worksheet

By default `xlsxwriter` will open to the first worksheet in the workbook. Worksheets are placed in the order in which we add them, so the `data` worksheet would appear first -- but we would like the `summary` worksheet instead. 

We can change what worksheet is activated upon opening with the `activate()` method:

In [56]:
# Activate the summary worksheet
# this was worksheet2 
worksheet2.activate()

Ready to see what we got?

In [57]:
 workbook.close()

# Drill

You can use the scaffolding below or try on your own. 

Create a blank worksheet in a workbook where the only cells that the user can modify are `A1:D1`.

Cell `A1` should be validated with a drop-down allowing a list of values `Hello`, `world` and `Python` and `C1` should contain a message alerting users that they can input integers between 1 and 10. 


In [41]:
import xlsxwriter
my_range = ['','','','']

workbook = xlsxwriter.Workbook('my-sheet.xlsx')
worksheet = workbook.add_worksheet()

# Protect the worksheet
worksheet.protect()

# Set up an unlocked cell format
unlocked = workbook.add_format({'locked':False})

# Write the range with an unprotected format
worksheet.write_row('A1', my_range, unlocked)

# Add data validation dropdown to cell A1
worksheet.data_validation('A1', {'validate':'list',
                                'source':['Hello','world','Python'],
                                'dropdown':True})

# Add data validation for integers to cell C1
worksheet.data_validation('C1', {'validate': 'integer',
'criteria': 'between',
'minimum': 1,
'maximum': 10,
'input_title': 'Enter an integer:',
'input_message': 'between 1 and 10'})


# Close the workbook
workbook.close()